In [22]:
import pandas as pd
import numpy as np
from util.config import versioned

filter_region = "10"  # Corresponds to wojewodztwo TERYT code

matched_all = versioned.read_parquet("matched.parquet")
# Filter people without any wojewodztwo info
matched_all = matched_all[matched_all["teryt_wojewodztwo"].notnull()]
# Get people with high enough scores
# TODO use the collision chance metric
good = matched_all["overall_score"] > 10.5
# Get people for the given region
local = matched_all["teryt_wojewodztwo"].apply(lambda row: filter_region in row)
local_good = matched_all[good & local]
# Filter out duplicates
local_good = local_good.drop_duplicates("krs_name").drop_duplicates("pkw_name")
f"Znaleziono {len(local_good)} osób"

'Znaleziono 491 osób'

In [23]:
local_good["krs_name"].nunique(), local_good["pkw_name"].nunique(), local_good["wiki_name"].nunique()

(491, 491, 9)

In [24]:
pd.set_option('display.max_rows', 600)
local_good

,overall_score,koryta_name,krs_name,pkw_name,wiki_name,birth_year,birth_date,employed_end,employed_krs,is_polityk,...,second_name_8,metaphone_9,birth_year_10,full_name_11,teryt_wojewodztwo,teryt_powiat,wiki_name_1,is_polityk_1,wiki_score,koryta_name_1
18,30.000000,Andrzej Melon,Andrzej Krzysztof Melon,MELON Andrzej,None,1957,1957-05-16,[2025-10-07T00:00:00.000000],[0000155205],None,...,,[MLN],1957,[MELON Andrzej],[10],[1063],None,None,NaN,Andrzej Melon
51,30.000000,Marek Rząsowski,Marek Rząsowski,Rząsowski Marek,None,1975,1975-10-22,"[2025-10-07T00:00:00.000000, 2024-09-24T00:00:...","[0000394569, 0000044561]",None,...,,"[RSSSK, RTSSFS]",1975,[Rząsowski Marek],[10],[1012],None,None,NaN,Marek Rząsowski
55,30.000000,Michał Franas,Michał Franas,FRANAS Michał,None,1980,1980-04-09,[2025-10-07T00:00:00.000000],[0000091303],None,...,,[FRNS],1980,[FRANAS Michał],[10],[1007],None,None,NaN,Michał Franas
57,30.000000,Michał Sobczak,Michał Jerzy Sobczak,SOBCZAK Michał,None,1981,1981-10-02,[2025-10-07T00:00:00.000000],"[0000392868, 0000014128]",None,...,,"[SPSK, SPXK]",1981,[SOBCZAK Michał],[10],[1005],None,None,NaN,Michał Sobczak
73,30.000000,Wojciech Brzeski,Wojciech Mariusz Brzeski,Brzeski Wojciech Mariusz,None,1963,1963-12-10,[2025-10-07T00:00:00.000000],[0000155205],None,...,mariusz,"[PRSSK, PRTSSK]",1963,[Brzeski Wojciech Mariusz],[10],[1020],None,None,NaN,Wojciech Brzeski
169,18.494024,None,Iwona Wieczorek,Wieczorek Iwona Małgorzata,Iwona Małgorzata Wieczorek,1967,1967-06-07,"[2015-02-25T00:00:00.000000, 2024-05-13T00:00:...","[0000114888, 0000056440]",Polityk,...,małgorzata,"[ASRK, FXRK]",1967,[Wieczorek Iwona Małgorzata],[10],[1020],Iwona Małgorzata Wieczorek,Polityk,1.494024,None
202,18.428899,None,Tomasz Sadzyński,Sadzyński Tomasz,Tomasz Sadzyński,1976,1976-11-12,"[2016-05-31T00:00:00.000000, 2025-10-07T00:00:...",[0000014128],Polityk,...,,"[STSSK, STTSSK]",1976,[Sadzyński Tomasz],[10],[1061],Tomasz Sadzyński,Polityk,1.428899,None
208,18.325818,None,Jolanta Maria Chełmińska,CHEŁMIŃSKA Jolanta,Jolanta Maria Chełmińska,1949,1949-03-08,[2006-08-30T00:00:00.000000],[0000022886],Polityk,...,,[XMSK],1949,[CHEŁMIŃSKA Jolanta],[10],"[1000, 1061]",Jolanta Maria Chełmińska,Polityk,1.325818,None
211,18.249046,None,Kajus Augustyniak,Augustyniak Kajus Mścisław,Kajus Mścisław Augustyniak,1962,1962-06-26,[2021-12-09T00:00:00.000000],[0000022886],Polityk,...,mścisław,[AKSTNK],1962,[Augustyniak Kajus Mścisław],[10],[1061],Kajus Mścisław Augustyniak,Polityk,1.249046,None
227,18.018213,None,Włodzimierz Tomaszewski,TOMASZEWSKI Włodzimierz,Włodzimierz Tomaszewski,1956,1956-11-16,[2019-12-06T00:00:00.000000],[0000134150],Biogram,...,,"[TMSSK, TMXFSK]",1956,[TOMASZEWSKI Włodzimierz],[10],[1061],Włodzimierz Tomaszewski,Biogram,1.518213,None


In [ ]:
# Write to output
local_output = versioned.get_path("people_lodzkie.csv")
local_good.to_csv(local_output,index=False)